In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator #
from tensorflow.keras.models import Sequential #
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D 
#Dense=computational unit make less
#Dropout = Numerical unit drop

In [15]:
#cd /path
#!unzip /path or archive.zip
num_classes = 131 #cat and dog
img_row = 64
img_col = 64 #image size change to matrics from square image
train_dataset = "/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training/"
test_dataset = "/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test/"

In [16]:
#data augmentation
train_datagen = ImageDataGenerator(
rescale = 1./255, #normaliation
rotation_range = 30,
width_shift_range = 0.4,
height_shift_range = 0.4,
horizontal_flip = True,
fill_mode='nearest'
)

In [17]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [18]:
train_generator = train_datagen.flow_from_directory(
    train_dataset,
    target_size=(img_row,img_col),
    batch_size = 32,
    class_mode = 'categorical', #multicalfication
    shuffle = True
)

In [19]:
validation_generator = train_datagen.flow_from_directory(
    test_dataset,
    target_size=(img_row,img_col),
    batch_size = 32, #if more batch fast
    class_mode = 'categorical', #categorical for multiimageclassification
    shuffle = True
)

In [20]:
model = Sequential() #feature map
#input layer for convolution
model.add(Conv2D(64,(3,3),padding='same',input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#hidden layer conv
model.add(Conv2D(32,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#model.add(Conv2D(16,(3,3),padding='same'))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size = (2,2)))
#model.add(Dropout(0.25))

#flatten conv
model.add(Flatten())#single layer 1D
model.add(Dense(512)) #only few send
model.add(Activation('relu'))
model.add(Dropout(0.5)) #size decrease

#neural network
model.add(Dense(units=512, activation = 'relu'))
model.add(Dense(units=256, activation = 'relu'))
model.add(Dense(units=256, activation = 'relu'))
model.add(Dense(131))
model.add(Activation('softmax')) #multiple output layer ma yes or no softmax do for probality
model.summary()

#output shape
# 64 by 64 image of 64 feature map

In [22]:
#comile before traing
#training
model.compile(loss='categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch = 6769//32,
    epochs = 5, #do maximum epoch for better test
    validation_data= validation_generator,
    validation_steps = 2268//32
)
model.save('fruits.h5')

In [24]:
#test
from tensorflow.python.keras.utils.generic_utils import class_and_config_for_serialized_keras_object
import numpy as np #to convert array
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

classifier = load_model('fruits.h5')
test_image = image.load_img('/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test/Nut Pecan/r_10_100.jpg',target_size = (64,64,3))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
result = classifier.predict(test_image)
print(result)
#if result > 0.6:
 # print("This is dog")
#else:
  #print("This is cat")

In [25]:
history.history

In [26]:
# Plotting our loss charts
import matplotlib.pyplot as plt

history_dict = history.history #

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)

line1 = plt.plot(epochs, val_loss_values, label='Validation/Test Loss')
line2 = plt.plot(epochs, loss_values, label='Training Loss')

plt.setp(line1, linewidth=2.0, marker = '+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker = '4', markersize=10.0)
plt.xlabel('Epochs') 
plt.ylabel('Loss')
plt.grid(True)
plt.legend()
plt.show()

In [27]:
# Plotting our acuracy charts
import matplotlib.pyplot as plt

history_dict = history.history

loss_values = history_dict['accuracy']
val_loss_values = history_dict['val_accuracy']
epochs = range(1, len(loss_values) + 1)

line1 = plt.plot(epochs, val_loss_values, label='Validation/Test Accuracy')
line2 = plt.plot(epochs, loss_values, label='Training Accuracy')
plt.setp(line1, linewidth=2.0, marker = '+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker = '4', markersize=10.0)
plt.xlabel('Epochs') 
plt.ylabel('Loss')
plt.grid(True)
plt.legend()
plt.show()